In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd
import numpy as np

In [ ]:
def clean_housing_price(df):
  # drop the extra columns and rows
  housing_price = df.drop(df.columns[range(62, len(df.columns))], axis=1)
  housing_price = housing_price.drop(list(range(385, 392)), axis=0)

  # build and insert month column and year column
  month = [date.split('-')[0] for date in housing_price['Mon-Yr']]
  year = [date.split('-')[1] for date in housing_price['Mon-Yr']]
  for idx, yr in enumerate(year):
    if yr[0] == '9':
      year[idx] = '19' + yr
    else:
      year[idx] = '20' + yr

  housing_price.insert(loc=1, value=month, column='Month')
  housing_price.insert(loc=2, value=year, column='Year')

  # format the price number notation
  for col in range(3, len(housing_price.columns)):
    for row in range(len(housing_price)):
      if not pd.isna(housing_price.iloc[row, col]):
        # remove the leading dollar sign and comma
        housing_price.iloc[row, col] = int(''.join(housing_price.iloc[row, col][1:].split(',')))
      else:
        # replace nan with None
        housing_price.iloc[row, col] = None

  return housing_price

In [ ]:
housing_price_raw = pd.read_csv('/content/drive/MyDrive/560 -Team Random/dataset/housing_price.csv')
housing_price = clean_housing_price(housing_price_raw)
housing_price.head()

,Mon-Yr,Month,Year,CA,Alameda,Amador,Butte,Calaveras,Contra-Costa,Del Norte,...,Yuba,Unnamed: 53,Condo,LA Metro,Central Coast,Central Valley,Far North,Inland Empire,S.F. Bay Area,SoCal
0,Jan-90,Jan,1990,194952,226149,None,102143,None,None,None,...,None,NaN,141519,203390,None,None,None,None,227366,None
1,Feb-90,Feb,1990,196273,219306,None,83333,None,None,None,...,None,NaN,144965,211024,None,None,None,None,234739,None
2,Mar-90,Mar,1990,194856,225162,None,100000,None,None,None,...,None,NaN,141132,209286,None,None,None,None,235337,None
3,Apr-90,Apr,1990,196111,229333,None,108000,None,None,None,...,None,NaN,145707,210302,None,None,None,None,233178,None
4,May-90,May,1990,195281,232291,None,100000,None,None,None,...,None,NaN,146060,210148,None,None,None,None,235881,None


In [ ]:
housing_price.set_index('Mon-Yr', inplace=True)

In [ ]:
v = housing_price.loc['Feb-90', 'Calaveras']
if v==None:
  print('fadsfa')

fadsfa


In [ ]:
date_list = list(housing_price['Mon-Yr'])
date_list

['Jan-90',
 'Feb-90',
 'Mar-90',
 'Apr-90',
 'May-90',
 'Jun-90',
 'Jul-90',
 'Aug-90',
 'Sep-90',
 'Oct-90',
 'Nov-90',
 'Dec-90',
 'Jan-91',
 'Feb-91',
 'Mar-91',
 'Apr-91',
 'May-91',
 'Jun-91',
 'Jul-91',
 'Aug-91',
 'Sep-91',
 'Oct-91',
 'Nov-91',
 'Dec-91',
 'Jan-92',
 'Feb-92',
 'Mar-92',
 'Apr-92',
 'May-92',
 'Jun-92',
 'Jul-92',
 'Aug-92',
 'Sep-92',
 'Oct-92',
 'Nov-92',
 'Dec-92',
 'Jan-93',
 'Feb-93',
 'Mar-93',
 'Apr-93',
 'May-93',
 'Jun-93',
 'Jul-93',
 'Aug-93',
 'Sep-93',
 'Oct-93',
 'Nov-93',
 'Dec-93',
 'Jan-94',
 'Feb-94',
 'Mar-94',
 'Apr-94',
 'May-94',
 'Jun-94',
 'Jul-94',
 'Aug-94',
 'Sep-94',
 'Oct-94',
 'Nov-94',
 'Dec-94',
 'Jan-95',
 'Feb-95',
 'Mar-95',
 'Apr-95',
 'May-95',
 'Jun-95',
 'Jul-95',
 'Aug-95',
 'Sep-95',
 'Oct-95',
 'Nov-95',
 'Dec-95',
 'Jan-96',
 'Feb-96',
 'Mar-96',
 'Apr-96',
 'May-96',
 'Jun-96',
 'Jul-96',
 'Aug-96',
 'Sep-96',
 'Oct-96',
 'Nov-96',
 'Dec-96',
 'Jan-97',
 'Feb-97',
 'Mar-97',
 'Apr-97',
 'May-97',
 'Jun-97',
 'Jul-97',

In [ ]:
def clean_water_quality(df):
  # pick and rename only the related columns
  water_quality = df[['COUNTY_NAME', 'SAMPLE_DATE', 'PARAMETER', 'RESULT', 'REPORTING_LIMIT']]
  water_quality.columns = ['County', 'Date', 'Parameter', 'Result', 'Limit']

  # format the date into month and year
  month_dict = {
    '01': 'Jan',
    '02': 'Feb',
    '03': 'Mar',
    '04': 'Apr',
    '05': 'May',
    '06': 'Jun',
    '07': 'Jul',
    '08': 'Aug',
    '09': 'Sep',
    '10': 'Oct',
    '11': 'Nov',
    '12': 'Dec'
  }
  month = [month_dict[date.split('/')[0]] for date in water_quality['Date']]
  year = [date.split(' ')[0].split('/')[2] for date in water_quality['Date']]

  # add date and year column and drop the original date column
  water_quality.insert(loc=2, value=month, column='Month')
  water_quality.insert(loc=3, value=year, column='Year')
  water_quality.drop(columns='Date', axis=1, inplace=True)

  # drop the rows with NaN result or limit value
  # becasue we cannot tell if this a polluted water source
  water_quality.dropna(subset=['Result', 'Limit'], inplace=True)
  water_quality['Polluted'] = water_quality['Result'] > water_quality['Limit']

  # calculate the result to limit ratio and handle the division by zero case
  water_quality['Ratio'] = water_quality['Result'] / water_quality['Limit']
  water_quality['Ratio'].fillna(0, inplace=True)

  return water_quality

In [4]:
water_quality_raw = pd.read_csv('/content/drive/MyDrive/560 -Team Random/dataset/lab_results.csv')
#water_quality = clean_water_quality(water_quality_raw)
water_quality_raw.head()

,STATION_ID,STATION_NAME,FULL_STATION_NAME,STATION_NUMBER,STATION_TYPE,LATITUDE,LONGITUDE,STATUS,COUNTY_NAME,SAMPLE_CODE,SAMPLE_DATE,SAMPLE_DEPTH,SAMPLE_DEPTH_UNITS,PARAMETER,RESULT,REPORTING_LIMIT,UNITS,METHOD_NAME
0,8135,01S04E32C001M,01S04E32C001M,01S04E32C001M,Groundwater,37.8073,-121.562,Review Status Unknown,Alameda,WDIS_0719152,05/03/1967 09:00,NaN,Feet,Conductance,3480.0,1.0,uS/cm,EPA 120.1
1,8135,01S04E32C001M,01S04E32C001M,01S04E32C001M,Groundwater,37.8073,-121.562,Review Status Unknown,Alameda,WDIS_0719152,05/03/1967 09:00,NaN,Feet,Dissolved Boron,7.7,0.1,mg/L,"Std Method 4500-B, C"
2,8135,01S04E32C001M,01S04E32C001M,01S04E32C001M,Groundwater,37.8073,-121.562,Review Status Unknown,Alameda,WDIS_0719152,05/03/1967 09:00,NaN,Feet,Dissolved Calcium,68.0,1.0,mg/L,EPA 215.2
3,8135,01S04E32C001M,01S04E32C001M,01S04E32C001M,Groundwater,37.8073,-121.562,Review Status Unknown,Alameda,WDIS_0719152,05/03/1967 09:00,NaN,Feet,Dissolved Chloride,758.0,0.1,mg/L,"Std Method 4500-Cl, B"
4,8135,01S04E32C001M,01S04E32C001M,01S04E32C001M,Groundwater,37.8073,-121.562,Review Status Unknown,Alameda,WDIS_0719152,05/03/1967 09:00,NaN,Feet,Dissolved Magnesium,59.0,0.1,mg/L,"Std Method 3500-Mg, E"


In [ ]:
air_quality_file_prefix = '/content/drive/MyDrive/dataset/annual_aqi_by_county_'
air_quality_dataset_list = []
for year in range(2000, 2022):
  air_quality_file_path = air_quality_file_prefix + str(year) + '.csv'
  curr_dataset = pd.read_csv(air_quality_file_path)
  curr_dataset = curr_dataset[curr_dataset['State'] == 'California']
  air_quality_dataset_list.append(curr_dataset)
air_quality = pd.concat(air_quality_dataset_list)
air_quality.head()

,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10
63,California,Alameda,2000,366,293,59,11,2,1,0,209,70,39,2,129,182,0,53,0
64,California,Amador,2000,366,238,83,34,11,0,0,192,112,44,15,0,351,0,0,0
65,California,Butte,2000,361,226,88,40,7,0,0,174,108,44,16,41,260,0,25,19
66,California,Calaveras,2000,364,245,71,34,14,0,0,200,115,38,29,0,311,0,23,1
67,California,Colusa,2000,365,306,54,5,0,0,0,105,61,36,0,0,280,0,30,55


In [ ]:
wells_data = pd.read_csv('/content/drive/MyDrive/dataset/wells_data.csv')
wells_data = wells_data.drop(['OID', 'API', 'LeaseName', 'WellNumber', 
                              'OperatorCo', 'OperatorNa', 'FieldName',
                              'AreaName', 'District', 'Section',
                              'Township', 'Range', 'BaseMeridi',
                              'GISSource'], axis = 1)
wells_data.head()

,WellStatus,WellType,WellTypeLa,CountyName,Latitude,Longitude,isConfiden,isDirectio,SpudDate,WellSymbol
0,Idle,DG,Dry Gas,Glenn,39.449158,-121.946289,N,N,02/17/1988,IdleDG
1,Idle,DG,Dry Gas,Glenn,39.440640,-121.951889,N,N,12/07/1999,IdleDG
2,Idle,DG,Dry Gas,Glenn,39.447174,-121.961273,N,N,05/13/2000,IdleDG
3,Idle,DG,Dry Gas,Glenn,39.443230,-121.951042,N,N,11/06/2002,IdleDG
4,Plugged,DH,Dry Hole,Glenn,39.438267,-121.947418,N,N,06/11/1959,PluggedDH


In [6]:
wells_data = pd.read_csv('/content/drive/MyDrive/560 -Team Random/dataset/wells_data.csv')

In [9]:
wells_data.head()

,OID,API,LeaseName,WellNumber,WellStatus,WellType,WellTypeLa,OperatorCo,OperatorNa,FieldName,...,Township,Range,BaseMeridi,Latitude,Longitude,GISSource,isConfiden,isDirectio,SpudDate,WellSymbol
0,-1,402120521,Angel Slough,2-1,Idle,DG,Dry Gas,C8720,"Crain Orchards, Inc.",Afton Gas,...,18N,01W,MD,39.449158,-121.946289,GPS,N,N,02/17/1988,IdleDG
1,-1,402120723,Kauai,3-1,Idle,DG,Dry Gas,R4085,"Royale Energy, Inc.",Afton Gas,...,18N,01W,MD,39.440640,-121.951889,GPS,N,N,12/07/1999,IdleDG
2,-1,402120734,Lanai,3-3,Idle,DG,Dry Gas,R4085,"Royale Energy, Inc.",Afton Gas,...,18N,01W,MD,39.447174,-121.961273,GPS,N,N,05/13/2000,IdleDG
3,-1,402120815,Anacapa,3-4,Idle,DG,Dry Gas,R4085,"Royale Energy, Inc.",Afton Gas,...,18N,01W,MD,39.443230,-121.951042,GPS,N,N,11/06/2002,IdleDG
4,-1,402100240,Max Pray-Central Valley-Boyd U,1,Plugged,DH,Dry Hole,03279,General Expl. Co. of Ca,Afton Gas,...,18N,01W,MD,39.438267,-121.947418,hud,N,N,06/11/1959,PluggedDH


In [8]:
wells_data.columns

Index(['OID', 'API', 'LeaseName', 'WellNumber', 'WellStatus', 'WellType',
       'WellTypeLa', 'OperatorCo', 'OperatorNa', 'FieldName', 'AreaName',
       'District', 'CountyName', 'Section', 'Township', 'Range', 'BaseMeridi',
       'Latitude', 'Longitude', 'GISSource', 'isConfiden', 'isDirectio',
       'SpudDate', 'WellSymbol'],
      dtype='object')